<a href="https://colab.research.google.com/github/DeepAIntelligence/TB-6CNN-FCLFCNN/blob/main/tb_6cnn_fclf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Packages


In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as trs
import torchvision.models as models
from torchsummary import summary
from torchvision.transforms.transforms import Grayscale
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import torch.utils.data as data
import os
import PIL as pil
from PIL import Image

from sklearn.metrics import RocCurveDisplay
from sklearn.preprocessing import LabelBinarizer
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder


In [ ]:
! pip install torchmetrics

In [ ]:
import torchmetrics
from torchmetrics.classification import MulticlassAUROC
from torchmetrics.functional.classification import multiclass_auroc
from torchmetrics import MetricCollection
from torchmetrics.classification import MulticlassAccuracy, MulticlassPrecision, MulticlassRecall

# Connect Google Colab To Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Define GPU

In [ ]:
dev = 'cuda' if torch.cuda.is_available() else 'cpu'
print(dev)

################################################################################
# Load Data
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

## Make Train,Val,Test Sets by ImagesNameList

In [ ]:
Book=pd.read_csv('/content/drive/MyDrive/ImagesList.csv')

tb_Book=Book[:800]
health_Book=Book[800:4600]
sick_Book=Book[4600:8400]

tb_Book = tb_Book.sample(frac=1,random_state=1).reset_index(drop=True)
health_Book = health_Book.sample(frac=1,random_state=1).reset_index(drop=True)
sick_Book = sick_Book.sample(frac=1,random_state=1).reset_index(drop=True)

tb_train=tb_Book[:396]
tb_val=tb_Book[396:524]
tb_test=tb_Book[524:800]
health_train=health_Book[:2280]
health_val=health_Book[2280:2888]
health_test=health_Book[2888:3800]
sick_train=sick_Book[:2280]
sick_val=sick_Book[2280:2888]
sick_test=sick_Book[2888:3800]

train_Book=pd.concat([tb_train,health_train,sick_train])
val_Book=pd.concat([tb_val,health_val,sick_val])
test_Book=pd.concat([tb_test,health_test,sick_test])

train_Book = train_Book.sample(frac=1,random_state=1).reset_index(drop=True)
val_Book = val_Book.sample(frac=1,random_state=1).reset_index(drop=True)
test_Book = test_Book.sample(frac=1,random_state=1).reset_index(drop=True)

# test_Book = test_Book [0:150]

print(train_Book)
print(val_Book)
print(test_Book)


## Define a Class of Custom DataLoader To Recieve and PreProcess Image by Name 

In [ ]:
class custom_data (data.Dataset):
  def __init__(self,root,book,size):
    self.book = book
    self.size = size
    self.root = root
    self.transform=trs.Compose([trs.ToPILImage(),
                                trs.Resize(256),
                                trs.ToTensor()])

  def _get(self):
    lst_img=[]
    images_lable=[]
    for idx in range(0,self.size):
      # print(idx)
      image_name=self.book.iloc[idx,0]
      image_class=self.book.iloc[idx,1]
      image_lable=self.book.iloc[idx,2]
      img=cv2.imread('%s/%s/%s.png'%(self.root, image_class,image_name))
      img=torch.tensor(img)
      img=img.permute(2,0,1)
      img=img.float()
      img=self.transform(img)
      img=torch.tensor(img,device=dev)
      image_lable=torch.tensor(image_lable,device=dev)
      imge=(img,image_lable)
      lst_img.append(imge)
      images_lable.append(image_lable)

    return lst_img ,images_lable

  def _remove(self):
    size=range(0,self.size)
    book = self.book.drop(index=size).reset_index(drop=True)
    return book

## Unzip Images To Call

In [ ]:
! unzip /content/drive/MyDrive/health.zip
! unzip /content/drive/MyDrive/sick.zip
! unzip /content/drive/MyDrive/tb.zip

################################################################################
# Define Models
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

## InceptionV3


In [ ]:
class convnet_inceptionv3(nn.Module):
  def __init__(self):
    super(convnet_inceptionv3,self).__init__()
    self.base = models.Inception3(aux_logits=False)
    self.base.fc=nn.Linear(in_features=2048,out_features=3,bias=True)
    # self.base.aux_logits=False

  def forward(self,x):
    y=self.base(x.clone())
    return y

model_inceptionv3 = convnet_inceptionv3().to(dev)
print(model_inceptionv3)
summary(model_inceptionv3,(3,256,256))


## ResNet50

In [ ]:
class convnet_res50(nn.Module):
  def __init__(self):
    super(convnet_res50,self).__init__()
    self.base = models.resnet50(pretrained=True)
    self.base.fc = nn.Linear(in_features=2048 ,out_features=3,bias=True)
    
  def forward(self,x):
    y=self.base(x.clone())
    return y

model_res50 = convnet_res50().to(dev)
print(model_res50)
summary(model_res50,(3,256,256))

## DenseNet201

In [ ]:
class convnet_dense201(nn.Module):
  def __init__(self):
    super(convnet_dense201,self).__init__()
    self.base = models.densenet201(pretrained=True)
    self.base.classifier = nn.Linear(in_features=1920 ,out_features=3,bias=True)
    
  def forward(self,x):
    y=self.base(x.clone())
    return y

model_dense201 = convnet_dense201().to(dev)
print(model_dense201)

model_parameters = filter(lambda p: p.requires_grad, model_dense201.parameters())
params = sum([np.prod(p.size()) for p in model_parameters])
print(params)

## MnasNet

In [ ]:
class convnet_mnas(nn.Module):
  def __init__(self):
    super(convnet_mnas,self).__init__()
    self.base = models.mnasnet1_0(pretrained=True)
    self.base.layers[1]=nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[4]=nn.BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[7]=nn.BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[8][0].layers[1]=nn.BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[8][0].layers[4]=nn.BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[8][0].layers[7]=nn.BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[8][1].layers[1]=nn.BatchNorm2d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[8][1].layers[4]=nn.BatchNorm2d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[8][1].layers[7]=nn.BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[8][2].layers[1]=nn.BatchNorm2d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[8][2].layers[4]=nn.BatchNorm2d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[8][2].layers[7]=nn.BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[9][0].layers[1]=nn.BatchNorm2d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[9][0].layers[4]=nn.BatchNorm2d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[9][0].layers[7]=nn.BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[9][1].layers[1]=nn.BatchNorm2d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[9][1].layers[4]=nn.BatchNorm2d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[9][1].layers[7]=nn.BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[9][2].layers[1]=nn.BatchNorm2d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[9][2].layers[4]=nn.BatchNorm2d(120, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[9][2].layers[7]=nn.BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[10][0].layers[1]=nn.BatchNorm2d(240, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[10][0].layers[4]=nn.BatchNorm2d(240, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[10][0].layers[7]=nn.BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[10][1].layers[1]=nn.BatchNorm2d(480, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[10][1].layers[4]=nn.BatchNorm2d(480, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[10][1].layers[7]=nn.BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[10][2].layers[1]=nn.BatchNorm2d(480, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[10][2].layers[4]=nn.BatchNorm2d(480, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[10][2].layers[7]=nn.BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[11][0].layers[1]=nn.BatchNorm2d(480, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[11][0].layers[4]=nn.BatchNorm2d(480, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[11][0].layers[7]=nn.BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[11][1].layers[1]=nn.BatchNorm2d(576, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[11][1].layers[4]=nn.BatchNorm2d(576, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[11][1].layers[7]=nn.BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[12][0].layers[1]=nn.BatchNorm2d(576, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[12][0].layers[4]=nn.BatchNorm2d(576, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[12][0].layers[7]=nn.BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[12][1].layers[1]=nn.BatchNorm2d(1152, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[12][1].layers[4]=nn.BatchNorm2d(1152, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[12][1].layers[7]=nn.BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[12][2].layers[1]=nn.BatchNorm2d(1152, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[12][2].layers[4]=nn.BatchNorm2d(1152, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[12][2].layers[7]=nn.BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[12][3].layers[1]=nn.BatchNorm2d(1152, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[12][3].layers[4]=nn.BatchNorm2d(1152, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[12][3].layers[7]=nn.BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[13][0].layers[1]=nn.BatchNorm2d(1152, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[13][0].layers[4]=nn.BatchNorm2d(1152, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[13][0].layers[7]=nn.BatchNorm2d(320, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    self.base.layers[15]=nn.BatchNorm2d(1280, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

    self.base.classifier[1] = nn.Linear(in_features=1280 ,out_features=3,bias=True)
    
  def forward(self,x):
    y=self.base(x.clone())
    return y

model_mnas = convnet_mnas().to(dev)
print(model_mnas)
summary(model_mnas,(3,256,256))

## MobilenetV3

In [ ]:
class convnet_mobilev3(nn.Module):
  def __init__(self):
    super(convnet_mobilev3,self).__init__()
    self.base = models.mobilenet_v3_large(pretrained=True)

    self.base.features[0][1]=nn.BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[1].block[0][1]=nn.BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[1].block[1][1]=nn.BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[2].block[0][1]=nn.BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[2].block[1][1]=nn.BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[2].block[2][1]=nn.BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[3].block[0][1]=nn.BatchNorm2d(72, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[3].block[1][1]=nn.BatchNorm2d(72, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[3].block[2][1]=nn.BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[4].block[0][1]=nn.BatchNorm2d(72, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[4].block[1][1]=nn.BatchNorm2d(72, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[4].block[3][1]=nn.BatchNorm2d(40, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[5].block[0][1]=nn.BatchNorm2d(120, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[5].block[1][1]=nn.BatchNorm2d(120, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[5].block[3][1]=nn.BatchNorm2d(40, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[6].block[0][1]=nn.BatchNorm2d(120, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[6].block[1][1]=nn.BatchNorm2d(120, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[6].block[3][1]=nn.BatchNorm2d(40, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[7].block[0][1]=nn.BatchNorm2d(240, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[7].block[1][1]=nn.BatchNorm2d(240, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[7].block[2][1]=nn.BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[8].block[0][1]=nn.BatchNorm2d(200, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[8].block[1][1]=nn.BatchNorm2d(200, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[8].block[2][1]=nn.BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[9].block[0][1]=nn.BatchNorm2d(184, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[9].block[1][1]=nn.BatchNorm2d(184, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[9].block[2][1]=nn.BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[10].block[0][1]=nn.BatchNorm2d(184, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[10].block[1][1]=nn.BatchNorm2d(184, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[10].block[2][1]=nn.BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[11].block[0][1]=nn.BatchNorm2d(480, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[11].block[1][1]=nn.BatchNorm2d(480, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[11].block[3][1]=nn.BatchNorm2d(112, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[12].block[0][1]=nn.BatchNorm2d(672, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[12].block[1][1]=nn.BatchNorm2d(672, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[12].block[3][1]=nn.BatchNorm2d(112, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[13].block[0][1]=nn.BatchNorm2d(672, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[13].block[1][1]=nn.BatchNorm2d(672, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[13].block[3][1]=nn.BatchNorm2d(160, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[14].block[0][1]=nn.BatchNorm2d(960, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[14].block[1][1]=nn.BatchNorm2d(960, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[14].block[3][1]=nn.BatchNorm2d(160, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[15].block[0][1]=nn.BatchNorm2d(960, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[15].block[1][1]=nn.BatchNorm2d(960, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[15].block[3][1]=nn.BatchNorm2d(160, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    self.base.features[16][1]=nn.BatchNorm2d(960, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)

    self.base.classifier[3]=nn.Linear(in_features=1280,out_features=3,bias=True)
  def forward(self,x):
    y=self.base(x.clone())
    return y

model_mobilev3 = convnet_mobilev3().to(dev)
print(model_mobilev3)
summary(model_mobilev3,(3,256,256))

## EfficientNet-B4

In [ ]:
class convnet_effi_b4(nn.Module):
  def __init__(self):
    super(convnet_effi_b4,self).__init__()
    self.base = models.efficientnet_b4(pretrained=True)
    self.base.classifier[1] = nn.Linear(in_features=1792 ,out_features=3,bias=True)
    
  def forward(self,x):
    y=self.base(x.clone())
    return y

model_effi_b4 = convnet_effi_b4().to(dev)
print(model_effi_b4)
summary(model_effi_b4,(3,256,256))

## Fully Connected Layer First




In [ ]:
# Define model
class NeuralNetwork_f(nn.Module):
    def __init__(self):
        super(NeuralNetwork_f, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3*256*256, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, 3*256*256)
        )

    def forward(self, x):
        x = self.flatten(x)
        y = self.linear_relu_stack(x)
        return y,x

model_f = NeuralNetwork_f().to(dev)
print(model_f)
summary(model_f,(3,256,256))

################################################################################
# Optimizing the Model Parameters
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||




**The optimizer for the defined networks should be run**



In [ ]:
loss_fn_mse = nn.MSELoss()
loss_fn_ce = nn.CrossEntropyLoss()

optimizer_inceptionv3 = torch.optim.Adam(model_inceptionv3.parameters(), lr=0.001)
optimizer_res50 = torch.optim.Adam(model_res50.parameters(), lr=0.001)
optimizer_res18 = torch.optim.Adam(model_res18.parameters(), lr=0.001)
optimizer_dense201 = torch.optim.Adam(model_dense201.parameters(), lr=0.001)
optimizer_mnas = torch.optim.Adam(model_mnas.parameters(), lr=0.001)
optimizer_mobilev3 = torch.optim.Adam(model_mobilev3.parameters(), lr=0.001)
optimizer_effi_b4 = torch.optim.Adam(model_effi_b4.parameters(), lr=0.001)
optimizer_f = torch.optim.Adam(model_f.parameters(), lr=0.001)


################################################################################
# Define Functions To Run
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

## Function To Training CNN

In [ ]:
def train_func(dataset, model, loss, optimizer, batch_size, epoch):
  vol=len(dataset)
  iter_s = vol // batch_size
  iter_f = vol % batch_size
  
  number_batch=0
  
  loss_all=[]
  yd_all=[]
  y_all=[]

  for idx in range(0,iter_s):
    number_batch=int(idx)+1
    print('epoch=' ,epoch ,'batch = ',number_batch)
    batch_images=custom_data(root='/content',book=dataset,size=batch_size)
    images,lables=batch_images._get()

    dataset=batch_images._remove()
    data_loader=data.DataLoader(dataset=images,batch_size=batch_size)

    for (x,y) in data_loader:
      model.train(True)
      # Compute prediction error
      yp=model(x)

      loss_value = loss(yp,y)

      yd=(torch.max(yp,1)[1])
      print('(',yd,')','|' , '(',y,')')

      num_corrects=torch.sum(torch.max(yp,1)[1]==y)
      acc = num_corrects.float()/float(batch_size)

      my_loss=loss_value
      my_yd=yd
      my_y=y
      loss_all.append(my_loss.detach().cpu().numpy().item())
      yd_all.append(my_yd.cpu().numpy())
      y_all.append(my_y.cpu().numpy())

      # Backpropagation
      optimizer.zero_grad()
      loss_value.backward()
      optimizer.step()

  
      print('num_corrects:' , num_corrects.item(),',   Train Loss:', loss_value.item(),
             ',   Train Accuracy:' , acc.item())
########################################################################################################
  if iter_f > 0 :
    print('epoch=' ,epoch ,'last batch = ' ,number_batch+1)
    batch_images=custom_data(root='/content',book=dataset,size=iter_f)
    images,lables=batch_images._get()
    dataset=batch_images._remove()
    data_loader=data.DataLoader(dataset=images,batch_size=iter_f)

    for (x,y) in data_loader:
      # Compute prediction error
      model.train(True)
      yp=model(x)

      loss_value = loss(yp,y)
      yd=(torch.max(yp,1)[1])
      print('(',yd,')','|' , '(',y,')')

      num_corrects=torch.sum(torch.max(yp,1)[1]==y)
      acc = num_corrects.float()/float(iter_f)

      my_loss=loss_value
      my_yd=yd
      my_y=y
      loss_all.append(my_loss.detach().cpu().numpy().item())
      yd_all.append(my_yd.cpu().numpy())
      y_all.append(my_y.cpu().numpy())

      # Backpropagation
      optimizer.zero_grad()
      loss_value.backward()
      optimizer.step()

      print('num_corrects:' , num_corrects.item(),',   Train Loss:', loss_value.item(),
              ',   Train Accuracy:' , acc.item())
      
  return loss_all , yd_all , y_all

## Function To Validation and Testing CNN

In [ ]:
def test_func(dataset, model, loss, batch_size, epoch):

  vol=len(dataset)
  iter_s = vol // batch_size
  iter_f = vol % batch_size
  
  number_batch=0
  
  loss_all=[]
  yd_all=[]
  y_all=[]
  yp_all=torch.zeros(0,3).to(dev)

  model.eval()

  loss_value=0
  for idx in range(0,iter_s):
    number_batch=int(idx)+1
    print('epoch=' ,epoch ,'batch = ',number_batch)
    batch_images=custom_data(root='/content',book=dataset,size=batch_size)
    images,lables=batch_images._get()
    dataset=batch_images._remove()
    data_loader=data.DataLoader(dataset=images,batch_size=batch_size)

    with torch.no_grad():
     for (x,y) in data_loader:
      model.train(True)
      # Compute prediction error
      yp=model(x)
      loss_value = loss(yp,y)

      yd=(torch.max(yp,1)[1])
      print('(',yd,')','|' , '(',y,')')

      num_corrects=torch.sum(torch.max(yp,1)[1]==y)
      acc = num_corrects.float()/float(batch_size)

      my_loss=loss_value
      my_yd=yd
      my_y=y
      loss_all.append(my_loss.detach().cpu().numpy().item())
      yd_all.append(my_yd.cpu().numpy())
      y_all.append(my_y.cpu().numpy())
      yp.to(dev)
      yp_all= torch.cat((yp_all, yp), 0)

      print('num_corrects:' , num_corrects.item(),',   val Loss:', loss_value.item(),
             ',   val Accuracy:' , acc.item())
########################################################################################################
  if iter_f > 0 :
    print('epoch=' ,epoch ,'last batch = ' ,number_batch+1)
    batch_images=custom_data(root='/content',book=dataset,size=iter_f)
    images,lables=batch_images._get()
    print('img=',torch.cuda.memory_allocated(device=dev))
    dataset=batch_images._remove()
    data_loader=data.DataLoader(dataset=images,batch_size=iter_f)

    with torch.no_grad():
     for (x,y) in data_loader:
      model.train(True)

      # Compute prediction error
      yp=model(x)
      loss_value = loss(yp,y)
      yd=(torch.max(yp,1)[1])
      print('(',yd,')','|' , '(',y,')')

      num_corrects=torch.sum(torch.max(yp,1)[1]==y)
      acc = num_corrects.float()/float(iter_f)

      my_loss=loss_value
      my_yd=yd
      my_y=y
      loss_all.append(my_loss.detach().cpu().numpy().item())
      yd_all.append(my_yd.cpu().numpy())
      y_all.append(my_y.cpu().numpy())
      yp.to(dev)
      yp_all= torch.cat((yp_all, yp), 0)

      print('num_corrects:' , num_corrects.item(),',   val Loss:', loss_value.item(),
              ',   val Accuracy:' , acc.item())
      
  return loss_all , yd_all , y_all , yp_all

## Function To Run CNN ,Save Models And Train&Val Results

In [ ]:
def run_func (name , model_fn , loss_fn , optimizer_fn , epoch , batch):

  tr_loss , tr_yd , tr_y =[],[],[]
  val_loss , val_yd , val_y =[],[],[]
  test_loss , test_yd , test_y =[],[],[]

  # train and val
  for epo in range(0,epoch):
    print(' ')
    print('epoch ========================================================================================= ',epo+1)
    print(' ')
    print('trainnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn')
    print(' ')

    tr_loss_e , tr_yd_e , tr_y_e = train_func(dataset=train_Book ,model=model_fn , loss=loss_fn ,optimizer=optimizer_fn ,batch_size=batch ,epoch=epo+1)
    print(' ')
    torch.save(model_fn.state_dict(), '/content/drive/MyDrive/modelsaved/saved_model_'+name+'_epoch='+str(epo+1)+'.pth')

    print('valllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllll')
    print(' ')

    val_loss_e , val_yd_e , val_y_e = test_func(dataset=val_Book ,model=model_fn , loss=loss_fn ,batch_size=batch,epoch=epo+1)

    tr_loss.append(tr_loss_e)
    tr_yd.append(tr_yd_e)
    tr_y.append(tr_y_e)
    val_loss.append(val_loss_e)
    val_yd.append(val_yd_e)
    val_y.append(val_y_e)

  np.savez('/content/drive/MyDrive/train_result_'+name   , tr_loss , tr_yd , tr_y)
  np.savez('/content/drive/MyDrive/val_result_'+name     , val_loss , val_yd , val_y)
  
  print('Done')

## Function To Load Models Parameters

In [ ]:
def load_model (model,name , epoch):

  model.load_state_dict(torch.load('/content/drive/MyDrive/modelsaved/saved_model_'+name+'_epoch='+str(epoch)+'.pth'))

  return model

## Function To Load Train&Val Results

In [ ]:
def func_load (name):
  train=np.load('/content/drive/MyDrive/train_result_'+name+'.npz',allow_pickle=True)
  val=np.load('/content/drive/MyDrive/val_result_'+name+'.npz',allow_pickle=True)

  sorted(train.files)
  a1=train['arr_0']
  a2=train['arr_1']
  a3=train['arr_2']

  sorted(val.files)
  b1=val['arr_0']
  b2=val['arr_1']
  b3=val['arr_2']

  all_result=[[a1,a2,a3],[b1,b2,b3]]
  return all_result 


## Function To plot Accuracy And Loss of Train&Val Results  

In [ ]:
def acc_mat_func (doc,number_train_img,numbatch_fortrain_loss,number_val_img,numbatch_forval_loss):
  name = func_load(doc)
  lst=name
  train=lst[0]
  val=lst[1]

  number_train_loss_epoch=len(train[0][0])
  number_train_epoch=len(train[1])
  number_train_batch=len(train[1][0])-1
  number_train_batchsize=len(train[1][0][0])
  number_train_endbatch= 4956 % number_train_batchsize
  if (number_train_endbatch==0):
    number_train_endbatch=16
  print('number_train_loss_epoch',number_train_loss_epoch)
  print('number_train_epoch',number_train_epoch)
  print('number_train_batch',number_train_batch)
  print('number_train_batchsize',number_train_batchsize)
  print('number_train_endbatch',number_train_endbatch)
  print('shape',train[0].shape)
  number_val_loss_epoch=len(val[0][0])
  number_val_epoch=len(val[1])
  number_val_batch=len(val[1][0])-1
  number_val_batchsize=len(val[1][0][0])
  number_val_endbatch= 1344 % number_val_batchsize
  if (number_val_endbatch==0):
    number_val_endbatch=16
  print('number_val_loss_epoch',number_val_loss_epoch)
  print('number_val_epoch',number_val_epoch)
  print('number_val_batch',number_val_batch)
  print('number_val_batchsize',number_val_batchsize)
  print('number_val_endbatch',number_val_endbatch)

#########################################################################     train
  print('################################################################     train     ###################################################################')

  train_loss=train[0]
  train_yd=[]
  train_y=[]

  batch_fix=0
  for epoch in range(0,number_train_epoch):
    for batch in range(0,number_train_batch):
      
      batch_fix=batch+1
      for size in range(0,number_train_batchsize):
        train_yd.append(train[1][epoch][batch][size])
        train_y.append(train[2][epoch][batch][size])

    for size in range(0,number_train_endbatch):
      train_yd.append(train[1][epoch][batch_fix][size])
      train_y.append(train[2][epoch][batch_fix][size])

  print('len(train_loss)',len(train_loss))
  print('len(train_yd)',len(train_yd))
  print('len(train_y)',len(train_y))

  train_acc_list=[]
  
  number_a_epoch=int(len(train_yd)/number_train_epoch)
  number_train_point=number_a_epoch // number_train_img
  number_train_end=int(len(train_yd)/number_train_epoch) % number_train_img
  print('number_a_epoch_train',number_a_epoch)
  print('number_train_epoch',number_train_epoch)
  print('number_train_point',number_train_point)
  print('number_train_end',number_train_end)

  for epoch in range(0,number_train_epoch):
    print('epoch',epoch)
    for point in range(0,int(number_train_point)):
      num_train=0
      for id in range(0,number_train_img):
        train_yd_b=torch.tensor(train_yd[(epoch*number_a_epoch)+(point*number_train_img)+id])
        train_y_b=torch.tensor(train_y[(epoch*number_a_epoch)+(point*number_train_img)+id])
        num_corrects_train=torch.sum(train_yd_b==train_y_b)
        num_train+=num_corrects_train
      acc_train=num_train/number_train_img*100
      print('num_train',num_train,'    number_train_img',number_train_img)

      print('acc_train',acc_train)
      train_acc_list.append(acc_train)
      print('train_acc_list.shape',len(train_acc_list))

    num_train=0
    if (number_train_end>0):
      for end in range (0,number_train_end):
        train_yd_b=torch.tensor(train_yd[(epoch*number_train_epoch)+(int(number_train_point)*number_train_img)+end])
        train_y_b=torch.tensor(train_y[(epoch*number_train_epoch)+(int(number_train_point)*number_train_img)+end])
        num_corrects_train=torch.sum(train_yd_b==train_y_b)
        num_train+=num_corrects_train
      acc_train=num_train/number_train_end*100
      print('num_train',num_train,'    number_train_end',number_train_end)
      print('acc_train',acc_train)
      train_acc_list.append(acc_train)
      print('train_acc_list.shape',len(train_acc_list))

    num_train=0

  train_loss_mean = []
  numloop_for_loss=int(number_train_loss_epoch // numbatch_fortrain_loss)
  endloop_for_loss=int(number_train_loss_epoch % numbatch_fortrain_loss)
  print('numbatch_fortrain_loss',numbatch_fortrain_loss)
  print('numloop_for_loss',numloop_for_loss)
  print('endloop_for_loss',endloop_for_loss)

  sumy=0
  mean=0  

  if (endloop_for_loss==0):
   for epoch in range(0,number_train_epoch):
    for loop in range(0, numloop_for_loss):
    
      for inloop in range(0,numbatch_fortrain_loss-1):
        losses_train = train_loss[epoch][(loop*numbatch_fortrain_loss)+inloop]*number_train_batchsize
        sumy = sumy + losses_train
      losses_train = train_loss[epoch][(loop*numbatch_fortrain_loss)+inloop+1]*number_train_endbatch
      sumy = sumy + losses_train
      mean = sumy / (((numbatch_fortrain_loss-1)*number_train_batchsize)+(number_train_endbatch))
      train_loss_mean.append(mean)
      sumy=0
      mean=0

  elif (endloop_for_loss>0):
   for epoch in range(0,number_train_epoch):
    for loop in range(0, numloop_for_loss):
      for inloop in range(0,numbatch_fortrain_loss):
        losses_train = train_loss[epoch][(loop*numbatch_fortrain_loss)+inloop]*number_train_batchsize
        sumy = sumy + losses_train
      mean = sumy / (numbatch_fortrain_loss*number_train_batchsize)
      train_loss_mean.append(mean)
      sumy=0
      mean=0
    loop+=1
    for endloop in range(0,endloop_for_loss-1):
      losses_train = train_loss[epoch][(loop*numbatch_fortrain_loss)+endloop]*number_train_batchsize
      sumy = sumy + losses_train
    losses_train = train_loss[epoch][(loop*numbatch_fortrain_loss)+endloop+1]*number_train_endbatch
    sumy = sumy + losses_train
    mean = sumy / (((numbatch_fortrain_loss-1)*number_train_batchsize)+(number_train_endbatch))
    train_loss_mean.append(mean)
    sumy=0
    mean=0
    
#########################################################################     val
  print('################################################################     val     ###################################################################')

  val_loss=val[0]
  val_yd=[]
  val_y=[]

  batch_fix=0
  for epoch in range(0,number_val_epoch):
    for batch in range(0,number_val_batch):
      
      batch_fix=batch+1
      for size in range(0,number_val_batchsize):
        val_yd.append(val[1][epoch][batch][size])
        val_y.append(val[2][epoch][batch][size])

    for size in range(0,number_val_endbatch):
      val_yd.append(val[1][epoch][batch_fix][size])
      val_y.append(val[2][epoch][batch_fix][size])

  print('len(val_loss)',len(val_loss))
  print('len(val_yd)',len(val_yd))
  print('len(val_y)',len(val_y))

  val_acc_list=[]
  
  number_a_epoch=int(len(val_yd)/number_val_epoch)
  number_val_point=number_a_epoch // number_val_img
  number_val_end=int(len(val_yd)/number_val_epoch) % number_val_img
  print('number_a_epoch_val',number_a_epoch)
  print('number_val_epoch',number_val_epoch)
  print('number_val_point',number_val_point)
  print('number_val_end',number_val_end)

  for epoch in range(0,number_val_epoch):
    print('epoch',epoch)
    for point in range(0,int(number_val_point)):
      num_val=0
      for id in range(0,number_val_img):
        val_yd_b=torch.tensor(val_yd[(epoch*number_a_epoch)+(point*number_val_img)+id])
        val_y_b=torch.tensor(val_y[(epoch*number_a_epoch)+(point*number_val_img)+id])
        num_corrects_val=torch.sum(val_yd_b==val_y_b)
        num_val+=num_corrects_val

      acc_val=num_val/number_val_img*100
      print('acc_val',acc_val)
      val_acc_list.append(acc_val)


    num_val=0
    if (number_val_end>0):
      for end in range (0,number_val_end):
        val_yd_b=torch.tensor(val_yd[(epoch*number_val_epoch)+(int(number_val_point)*number_val_img)+end])
        val_y_b=torch.tensor(val_y[(epoch*number_val_epoch)+(int(number_val_point)*number_val_img)+end])
        num_corrects_val=torch.sum(val_yd_b==val_y_b)
        num_val+=num_corrects_val
      acc_val=num_val/number_val_end*100
      print('acc_val',acc_val)
      val_acc_list.append(acc_val)
    num_val=0


  val_loss_mean = []
  numloop_for_loss=int(number_val_loss_epoch // numbatch_forval_loss)
  endloop_for_loss=int(number_val_loss_epoch % numbatch_forval_loss)
  print('numbatch_forval_loss',numbatch_forval_loss)
  print('numloop_for_loss',numloop_for_loss)
  print('endloop_for_loss',endloop_for_loss)

  sumy=0
  mean=0  

  if (endloop_for_loss==0):
   for epoch in range(0,number_val_epoch):
    for loop in range(0, numloop_for_loss):
    
      for inloop in range(0,numbatch_forval_loss-1):
        losses_val = val_loss[epoch][(loop*numbatch_forval_loss)+inloop]*number_val_batchsize
        sumy = sumy + losses_val
      losses_val = val_loss[epoch][(loop*numbatch_forval_loss)+inloop+1]*number_val_endbatch
      sumy = sumy + losses_val
      mean = sumy / (((numbatch_forval_loss-1)*number_val_batchsize)+(number_val_endbatch))
      val_loss_mean.append(mean)
      print('  mean=',mean , '  val_loss_mean.shape',len(val_loss_mean))
      sumy=0
      mean=0

  elif (endloop_for_loss>0):
   for epoch in range(0,number_val_epoch):
    for loop in range(0, numloop_for_loss):
      for inloop in range(0,numbatch_forval_loss):
        losses_val = val_loss[epoch][(loop*numbatch_forval_loss)+inloop]*number_val_batchsize
        sumy = sumy + losses_val
      mean = sumy / (numbatch_forval_loss*number_val_batchsize)
      val_loss_mean.append(mean)
      print('  mean=',mean , '  val_loss_mean.shape',len(val_loss_mean))
      sumy=0
      mean=0
    loop+=1
    for endloop in range(0,endloop_for_loss-1):
      losses_val = val_loss[epoch][(loop*numbatch_forval_loss)+endloop]*number_val_batchsize
      sumy = sumy + losses_val
    losses_val = val_loss[epoch][(loop*numbatch_forval_loss)+endloop+1]*number_val_endbatch
    sumy = sumy + losses_val
    mean = sumy / (((numbatch_forval_loss-1)*number_val_batchsize)+(number_val_endbatch))
    val_loss_mean.append(mean)
    print('  mean=',mean , '  val_loss_mean.shape',len(val_loss_mean))
    sumy=0
    mean=0

  return train_acc_list,  train_loss_mean,    val_acc_list,  val_loss_mean  

## Function To Save The Test Results And Plot Confusion Matrix

In [ ]:
def final_test_model (model_fn,name,epo_type,data,loss_fn,batch_size):

  model=load_model (model_fn ,name,epo_type)

  test_loss_h , test_yd_h , test_y_h , test_yp_h =[],[],[],[]
  test_loss_e , test_yd_e , test_y_e , test_yp_e = test_func(dataset=data ,model=model , loss=loss_fn ,batch_size=batch_size,epoch=1)

  test_loss_h.append(test_loss_e)
  test_yd_h.append(test_yd_e)
  test_y_h.append(test_y_e)

 ##########################################################################    test
  print('#################################################################    test   ###################################################################') 

  number_test_loss_epoch=len(test_loss_h[0])
  number_test_epoch=len(test_yd_h)
  number_test_batch=len(test_yd_h[0])-1
  number_test_batchsize=batch_size
  number_test_endbatch= len(data) % number_test_batchsize
  if (number_test_endbatch==0):
    number_test_endbatch=16

  test_loss=[]
  test_yd=[]
  test_y=[]

  batch_fix=0
  for epoch in range(0,1):
    for batch in range(0,number_test_batch):
      
      batch_fix=batch+1
      for size in range(0,number_test_batchsize):
        test_yd.append(test_yd_h[epoch][batch][size])
        test_y.append(test_y_h[epoch][batch][size])

    for size in range(0,number_test_endbatch):
      test_yd.append(test_yd_h[epoch][batch_fix][size])
      test_y.append(test_y_h[epoch][batch_fix][size])

  number_a_epoch=1
  number_test_point=1

  test_matrix=torch.tensor([[0,0,0],[0,0,0],[0,0,0]])

  num_test=0
  for id in range(0,len(test_yd)):
    test_yd_b=torch.tensor(test_yd[id])
    test_y_b=torch.tensor(test_y[id])
    # Matrix
    if(test_yd_b==0 and test_y_b==0):
      test_matrix[0][0]+=1;
    elif(test_yd_b==0 and test_y_b==1):
      test_matrix[0][1]+=1;
    elif(test_yd_b==0 and test_y_b==2):
      test_matrix[0][2]+=1;
    elif(test_yd_b==1 and test_y_b==0):
      test_matrix[1][0]+=1;
    elif(test_yd_b==1 and test_y_b==1):
      test_matrix[1][1]+=1;
    elif(test_yd_b==1 and test_y_b==2):
      test_matrix[1][2]+=1;
    elif(test_yd_b==2 and test_y_b==0):
      test_matrix[2][0]+=1;
    elif(test_yd_b==2 and test_y_b==1):
      test_matrix[2][1]+=1;
    elif(test_yd_b==2 and test_y_b==2):
      test_matrix[2][2]+=1;

    num_corrects_test=torch.sum(test_yd_b==test_y_b)
    num_test+=num_corrects_test
    acc_test=num_test/len(test_yd)*100

  sumy=0
  mean=0

  for id in range(0,number_test_batch):
    losses_test = test_loss_h[0][id]*batch_size
    sumy = sumy + losses_test

  if (number_test_endbatch>0):
    losses_test = test_loss_h[0][id+1]*number_test_endbatch
    sumy = sumy + losses_test

  mean = sumy / len(test_yd)
  test_loss_all=mean



  soft = nn.Softmax(dim=1)
  test_yp_e=soft(test_yp_e)
  test_y = torch.Tensor(test_y)


  test_yp_e = (test_yp_e).to('cpu')
  test_yp_e=test_yp_e.float()
  test_y = test_y.to('cpu')
  test_y=test_y.int()

  auroc = MulticlassAUROC(task="multiclass" ,num_classes=3, average=None, thresholds=None)
  print('AUC',auroc(test_yp_e, test_y))
  auroc = MulticlassAUROC(task="multiclass" ,num_classes=3, average="macro", thresholds=None)
  print('AUC',auroc(test_yp_e, test_y))

  metric_collection = MetricCollection([
    MulticlassAccuracy(num_classes=3, average="macro"),
    MulticlassPrecision(num_classes=3, average="micro"),
    MulticlassRecall(num_classes=3, average="micro")
  ])
  print(metric_collection(test_yp_e, test_y))

  metric_collection = MetricCollection([
    MulticlassAccuracy(num_classes=3, average=None),
    MulticlassPrecision(num_classes=3, average=None),
    MulticlassRecall(num_classes=3, average=None)
  ])
  print(metric_collection(test_yp_e, test_y))

  print('num_test',num_test)
  print('acc_test',acc_test)
  print('test_loss_all',test_loss_all)
  print('test_matrix')
  print(test_matrix)
  


################################################################################
# Run CNNs 
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

**The line related to the defined networks should be executed**

In [ ]:
run_func('inceptionv3_new' , model_inceptionv3 , loss_fn_ce , optimizer_inceptionv3 , 20 , 16)
run_func('res50_result' , model_res50 , loss_fn_ce , optimizer_res50 , 20 , 16)
run_func('dense201_result' , model_dense201 , loss_fn_ce , optimizer_dense201 , 20 , 16)
run_func('mnas_result' , model_mnas , loss_fn_ce , optimizer_mnas ,20 , 16)
run_func('mobilev3_result' , model_mobilev3 , loss_fn_ce , optimizer_mobile , 20 , 16)
run_func('efficient_b4_result' , model_effi_b4 , loss_fn_ce , optimizer_effi_b4 , 20 , 16)


################################################################################
# Plot CNN Results
The first step is to Load training and evaluation results. In training, accuracy is computing for each 1024 of images and Loss is computing for each 104 batches, and in Validation, accuracy is computing for each 1344 of images and Loss is computing for each 84 batches.
The second step is to Plot the accuracy of the training and evaluation
The third step is to plot the Loss of training and evaluation
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

## InceptionV3

### Load Training And Evaluation Results

In [ ]:
train_acc_inc , train_loss_inc , val_acc_inc , val_loss_inc = acc_mat_func ('inceptionv3_result',1239,104,1344,84,0)

### Plot Accuracy

In [ ]:
print(len(train_acc_inc))
print(len(val_acc_inc))
plt.plot(range(1,len(train_acc_inc)+1),train_acc_inc)
s_acc=(len(train_acc_inc))/(len(val_acc_inc))
a_acc=2.5
d_acc=[2.5]
for id in range(1,len(val_acc_inc)):
  d_acc.append(a_acc+s_acc)
  a_acc+=s_acc
plt.plot(d_acc,val_acc_inc)
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.ylim(0,101)
plt.xticks([2.5,6.5,10.5,14.5,18.5,22.5,26.5,30.5,34.5,38.5,42.5,46.5,50.5,54.5,58.5,62.5,66.5,70.5,74.5,78.5],
           ['1', '2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'])
plt.legend(['train','val'])

### Plot Loss 

In [ ]:
plt.plot(range(1,len(train_loss_inc)+1),train_loss_inc)
print(len(train_loss_inc))
print(len(val_loss_inc))
s_loss=(len(train_loss_inc))/(len(val_loss_inc))
a_loss=2
d_loss=[2]
for id in range(1,len(val_loss_inc)):
  d_loss.append(a_loss+s_loss)
  a_loss+=s_loss
plt.plot(d_loss,val_loss_inc)
plt.xlabel('epochs')
plt.ylabel('loss')
plt.xticks([2,5,8,11,14,17,20,23,26,29,32,35,38,41,44,47,50,53,56,59],
           ['1', '2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'])
plt.ylim(0,5)
plt.legend(['train','val'])

## ResNet50

### Load Training And Evaluation Results

In [ ]:
train_acc_res , train_loss_res , val_acc_res , val_loss_res = acc_mat_func ('res50_result',1239,104,1344,84,0)

### Plot Accuracy

In [ ]:
print(len(train_acc_res))
print(len(val_acc_res))
plt.plot(range(1,len(train_acc_res)+1),train_acc_res)
s_acc=(len(train_acc_res))/(len(val_acc_res))
a_acc=2.5
d_acc=[2.5]
for id in range(1,len(val_acc_res)):
  d_acc.append(a_acc+s_acc)
  a_acc+=s_acc
plt.plot(d_acc,val_acc_res)
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.ylim(0,101)
plt.xticks([2.5,6.5,10.5,14.5,18.5,22.5,26.5,30.5,34.5,38.5,42.5,46.5,50.5,54.5,58.5,62.5,66.5,70.5,74.5,78.5],
           ['1', '2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'])
plt.legend(['train','val'])

### Plot Loss 

In [ ]:
plt.plot(range(1,len(train_loss_res)+1),train_loss_res)
print(len(train_loss_res))
print(len(val_loss_res))
s_loss=(len(train_loss_res))/(len(val_loss_res))
a_loss=2
d_loss=[2]
for id in range(1,len(val_loss_res)):
  d_loss.append(a_loss+s_loss)
  a_loss+=s_loss
plt.plot(d_loss,val_loss_res)
plt.xlabel('epochs')
plt.ylabel('loss')
plt.xticks([2,5,8,11,14,17,20,23,26,29,32,35,38,41,44,47,50,53,56,59],
           ['1', '2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'])
plt.ylim(0,5)
plt.legend(['train','val'])

## DenseNet201

### Load Training And Evaluation Results

In [ ]:
train_acc_dense , train_loss_dense , val_acc_dense , val_loss_dense = acc_mat_func ('dense201_result',1239,104,1344,84,0)

### Plot Accuracy

In [ ]:
print(len(train_acc_dense))
print(len(val_acc_dense))
plt.plot(range(1,len(train_acc_dense)+1),train_acc_dense)
s_acc=(len(train_acc_dense))/(len(val_acc_dense))
a_acc=2.5
d_acc=[2.5]
for id in range(1,len(val_acc_dense)):
  d_acc.append(a_acc+s_acc)
  a_acc+=s_acc
plt.plot(d_acc,val_acc_dense)
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.ylim(0,101)
plt.xticks([2.5,6.5,10.5,14.5,18.5,22.5,26.5,30.5,34.5,38.5,42.5,46.5,50.5,54.5,58.5,62.5,66.5,70.5,74.5,78.5],
           ['1', '2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'])
plt.legend(['train','val'])

### Plot Loss 

In [ ]:
plt.plot(range(1,len(train_loss_dense)+1),train_loss_dense)
print(len(train_loss_dense))
print(len(val_loss_dense))
s_loss=(len(train_loss_dense))/(len(val_loss_dense))
a_loss=2
d_loss=[2]
for id in range(1,len(val_loss_dense)):
  d_loss.append(a_loss+s_loss)
  a_loss+=s_loss
plt.plot(d_loss,val_loss_dense)
plt.xlabel('epochs')
plt.ylabel('loss')
plt.xticks([2,5,8,11,14,17,20,23,26,29,32,35,38,41,44,47,50,53,56,59],
           ['1', '2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'])
plt.ylim(0,5)
plt.legend(['train','val'])

## MnasNet

### Load Training And Evaluation Results

In [ ]:
train_acc_mnas , train_loss_mnas , val_acc_mnas , val_loss_mnas = acc_mat_func ('mnas_result',1239,104,1344,84,0)

### Plot Accuracy

In [ ]:
print(len(train_acc_mnas))
print(len(val_acc_mnas))
plt.plot(range(1,len(train_acc_mnas)+1),train_acc_mnas)
s_acc=(len(train_acc_mnas))/(len(val_acc_mnas))
a_acc=2.5
d_acc=[2.5]
for id in range(1,len(val_acc_mnas)):
  d_acc.append(a_acc+s_acc)
  a_acc+=s_acc
plt.plot(d_acc,val_acc_mnas)
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.ylim(0,101)
plt.xticks([2.5,6.5,10.5,14.5,18.5,22.5,26.5,30.5,34.5,38.5,42.5,46.5,50.5,54.5,58.5,62.5,66.5,70.5,74.5,78.5],
           ['1', '2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'])
plt.legend(['train','val'])

### Plot Loss 

In [ ]:
plt.plot(range(1,len(train_loss_mnas)+1),train_loss_mnas)
print(len(train_loss_mnas))
print(len(val_loss_mnas))
s_loss=(len(train_loss_mnas))/(len(val_loss_mnas))
a_loss=2
d_loss=[2]
for id in range(1,len(val_loss_mnas)):
  d_loss.append(a_loss+s_loss)
  a_loss+=s_loss
plt.plot(d_loss,val_loss_mnas)
plt.xlabel('epochs')
plt.ylabel('loss')
plt.xticks([2,5,8,11,14,17,20,23,26,29,32,35,38,41,44,47,50,53,56,59],
           ['1', '2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'])
plt.ylim(0,5)
plt.legend(['train','val'])

## MobileNetV3

### Load Training And Evaluation Results

In [ ]:
train_acc_mob , train_loss_mob , val_acc_mob , val_loss_mob = acc_mat_func ('mobilev3_result',1239,104,1344,84,0)

### Plot Accuracy

In [ ]:
print(len(train_acc_mob))
print(len(val_acc_mob))
plt.plot(range(1,len(train_acc_mob)+1),train_acc_mob)
s_acc=(len(train_acc_mob))/(len(val_acc_mob))
a_acc=2.5
d_acc=[2.5]
for id in range(1,len(val_acc_mob)):
  d_acc.append(a_acc+s_acc)
  a_acc+=s_acc
plt.plot(d_acc,val_acc_mob)
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.ylim(0,101)
plt.xticks([2.5,6.5,10.5,14.5,18.5,22.5,26.5,30.5,34.5,38.5,42.5,46.5,50.5,54.5,58.5,62.5,66.5,70.5,74.5,78.5],
           ['1', '2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'])
plt.legend(['train','val'])

### Plot Loss 

In [ ]:
plt.plot(range(1,len(train_loss_mob)+1),train_loss_mob)
print(len(train_loss_mob))
print(len(val_loss_mob))
s_loss=(len(train_loss_mob))/(len(val_loss_mob))
a_loss=2
d_loss=[2]
for id in range(1,len(val_loss_mob)):
  d_loss.append(a_loss+s_loss)
  a_loss+=s_loss
plt.plot(d_loss,val_loss_mob)
plt.xlabel('epochs')
plt.ylabel('loss')
plt.xticks([2,5,8,11,14,17,20,23,26,29,32,35,38,41,44,47,50,53,56,59],
           ['1', '2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'])
plt.ylim(0,5)
plt.legend(['train','val'])

## EfficientNet_B4

### Load Training And Evaluation Results

In [ ]:
train_acc_effi , train_loss_effi , val_acc_effi , val_loss_effi = acc_mat_func ('efficient_b4_result',1239,104,1344,84)

### Plot Accuracy

In [ ]:
print(len(train_acc_effi))
print(len(val_acc_effi))
plt.plot(range(1,len(train_acc_effi)+1),train_acc_effi)
s_acc=(len(train_acc_effi))/(len(val_acc_effi))
a_acc=2.5
d_acc=[2.5]
for id in range(1,len(val_acc_effi)):
  d_acc.append(a_acc+s_acc)
  a_acc+=s_acc
plt.plot(d_acc,val_acc_effi)
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.ylim(0,101)
plt.xticks([2.5,6.5,10.5,14.5,18.5,22.5,26.5,30.5,34.5,38.5,42.5,46.5,50.5,54.5,58.5,62.5,66.5,70.5,74.5,78.5],
           ['1', '2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'])
plt.legend(['train','val'])

### Plot Loss 

In [ ]:
plt.plot(range(1,len(train_loss_effi)+1),train_loss_effi)
s_loss=(len(train_loss_effi))/(len(val_loss_effi))
a_loss=2
d_loss=[2]
for id in range(1,len(val_loss_effi)):
  d_loss.append(a_loss+s_loss)
  a_loss+=s_loss
plt.plot(d_loss,val_loss_effi)
plt.xlabel('epochs')
plt.ylabel('loss')
plt.xticks([2,5,8,11,14,17,20,23,26,29,32,35,38,41,44,47,50,53,56,59],
           ['1', '2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'])
plt.ylim(0,5)
plt.legend(['train','val'])

## All Networks in A Chart

### Plot Accuracy

In [ ]:
s_acc=(len(train_acc_effi))/(len(val_acc_effi))
a_acc=2.5
d_acc=[2.5]
for id in range(1,len(val_acc_effi)):
  d_acc.append(a_acc+s_acc)
  a_acc+=s_acc
plt.plot(d_acc,val_acc_inc)
plt.plot(d_acc,val_acc_res)
plt.plot(d_acc,val_acc_dense)
plt.plot(d_acc,val_acc_mnas)
plt.plot(d_acc,val_acc_mob)
plt.plot(d_acc,val_acc_effi)
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.ylim(80,101)
plt.xticks([2.5,6.5,10.5,14.5,18.5,22.5,26.5,30.5,34.5,38.5,42.5,46.5,50.5,54.5,58.5,62.5,66.5,70.5,74.5,78.5],
           ['1', '2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'])
plt.legend(['InceptionV3','ResNet50','DenseNet201','MnasNet','MobileNetV3','EfficientNet_B4'])

### Plot Loss 

In [ ]:
s_loss=(len(train_loss_effi))/(len(val_loss_effi))
a_loss=2
d_loss=[2]
for id in range(1,len(val_loss_effi)):
  d_loss.append(a_loss+s_loss)
  a_loss+=s_loss
plt.plot(d_loss,val_loss_inc)
plt.plot(d_loss,val_loss_res)
plt.plot(d_loss,val_loss_dense)
plt.plot(d_loss,val_loss_mnas)
plt.plot(d_loss,val_loss_mob)
plt.plot(d_loss,val_loss_effi)
plt.xlabel('epochs')
plt.ylabel('loss')
plt.xticks([2,5,8,11,14,17,20,23,26,29,32,35,38,41,44,47,50,53,56,59],
           ['1', '2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'])
plt.ylim(0,1)
plt.legend(['InceptionV3','ResNet50','DenseNet201','MnasNet','MobileNetV3','EfficientNet_B4'])


################################################################################
# Test Models And Plot Confusion Matrix
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

**The line related to the defined networks should be executed**

In [ ]:
final_test_model (model_inceptionv3,'inception_RGB_256',12,test_Book,loss_fn_ce,16)
final_test_model (model_res50,'res50_256',20,test_Book,loss_fn_ce,16)
final_test_model (model_dense201,'dense201_256',18,test_Book,loss_fn_ce,16)
final_test_model (model_mnas,'mnas_0.1_RGB_256',20,test_Book,loss_fn_ce,16)
final_test_model (model_mobilev3,'mobilev3_large_256',17,test_Book,loss_fn_ce,16)
final_test_model (model_effi_b4,'efficient_b4_256',19,test_Book,loss_fn_ce,16)

################################################################################
# Define Functions For Fully Connected Layer First And Run And Save Results
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

## Function To Training Fully Connected Layer First

In [ ]:
def train_f(dataset, model, loss ,optimizer,batch_size,epoch):

  vol=len(dataset)
  iter_s = vol // batch_size
  iter_f = vol % batch_size
  s=[]
  t=[]
  l=[]
  lossy=[]

  for idx in range(0,iter_s):
    number_batch=int(idx)+1
    print('epoch=' ,epoch ,'batch = ',number_batch)
    batch_images=custom_data(root='/content',book=dataset,size=batch_size)
    images,lables=batch_images._get()
    dataset=batch_images._remove()
    data_loader=data.DataLoader(dataset=images,batch_size=batch_size)

    for (X,y) in data_loader:
        model.train
        X, y = X.to(dev), y.to(dev)

        # Compute prediction error
        pred,x = model(X)
        loss_value = loss(pred, x)

        # Backpropagation
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()
        lossy.append(loss_value.item())
        
        print('   Train Loss:', loss_value.item())    
    ########################################################################################################
  if iter_f > 0 :
    print('epoch=' ,epoch ,'last batch = ' ,number_batch+1)
    batch_images=custom_data(root='/content',book=dataset,size=iter_f)
    images,lables=batch_images._get()
    dataset=batch_images._remove()
    data_loader=data.DataLoader(dataset=images,batch_size=iter_f)

    for (X,y) in data_loader:
        model.train
        X, y = X.to(dev), y.to(dev)

        # Compute prediction error
        pred,x = model(X)
        loss_value = loss(pred, x)

        # Backpropagation
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()
        lossy.append(loss_value.item())
        s.append(x.cpu().detach().numpy())
        t.append(pred.cpu().detach().numpy())
        l.append(y.cpu().detach().numpy())
        
        print('   Train Loss:', loss_value.item())  

  return s , t ,l, lossy

## Function To Validation and Testing Fully Connected Layer First

In [ ]:
def test_f(dataset, model, loss,batch_size,epoch):

  model.eval()
  vol=len(dataset)
  iter_s = vol // batch_size
  iter_f = vol % batch_size
  s=[]
  t=[]
  l=[]
  loss_all=[]

  for idx in range(0,iter_s):
    number_batch=int(idx)+1
    print('epoch=' ,epoch ,'batch = ',number_batch)
    batch_images=custom_data(root='/content',book=dataset,size=batch_size)
    images,lables=batch_images._get()
    dataset=batch_images._remove()
    data_loader=data.DataLoader(dataset=images,batch_size=batch_size)

    with torch.no_grad():
      for (X,y) in data_loader:
        # Compute prediction error
        pred,x = model(X)
        loss_value = loss(pred,x)
        loss_all.append(loss_value.detach().cpu().numpy().item())
        print('   val Loss:', loss_value.item())

    ########################################################################################################
  if iter_f > 0 :
    print('epoch=' ,epoch ,'last batch = ' ,number_batch+1)
    batch_images=custom_data(root='/content',book=dataset,size=iter_f)
    images,lables=batch_images._get()
    dataset=batch_images._remove()
    data_loader=data.DataLoader(dataset=images,batch_size=iter_f)

    with torch.no_grad():
      for (X,y) in data_loader:
        # Compute prediction error

        pred,x = model(X)
        loss_value = loss(pred,x)
        loss_all.append(loss_value.detach().cpu().numpy().item())
        s.append(x.cpu().detach().numpy())
        t.append(pred.cpu().detach().numpy())
        l.append(y.cpu().detach().numpy())
        print('   val Loss:', loss_value.item())

  return s , t ,l, loss_all

## Run Fully Connected Layer First And Save Results

In [ ]:
tr_f_loss=[]
val_f_loss=[]
for epo in range(0,20):
    
    print(f"Epoch {epo+1}\n-------------------------------")
    x_tr, y_tr, l_tr, loss_tr = train_f(train_Book, model_f, loss_fn_mse, optimizer_f,16,epo+1)
    tr_f_loss.append(loss_tr)

    print('valllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllll')
    x_val, y_val, l_val, loss_val = test_f(val_Book, model_f, loss_fn_mse ,16,epo+1)
    val_f_loss.append(loss_val)

torch.save(model_f.state_dict(), '/content/drive/MyDrive/modelsaved/saved_model_model_f'+str(epo+1)+'.pth')

np.save('/content/drive/MyDrive/model_f_loss_train'   , tr_f_loss)
np.save('/content/drive/MyDrive/model_f_loss_val'   , val_f_loss)

np.save('/content/drive/MyDrive/model_f_x_train'   , x_tr)
np.save('/content/drive/MyDrive/model_f_y_train'   , y_tr)
np.save('/content/drive/MyDrive/model_f_l_train'   , l_tr)

np.save('/content/drive/MyDrive/model_f_x_val'   , x_val)
np.save('/content/drive/MyDrive/model_f_y_val'   , y_val)
np.save('/content/drive/MyDrive/model_f_l_val'   , l_val)

print('Done')

## Plot Result of Fully Connected Layer First

In [ ]:
tr_ls=np.load('/content/drive/MyDrive/model_f_loss_train.npy',allow_pickle=True)
val_ls=np.load('/content/drive/MyDrive/model_f_loss_val.npy',allow_pickle=True)

t=tr_ls
v=val_ls

tiri=[]
vali=[]
for i in range(0,20):
  for j in range(0,310):
    tiri.append(t[i][j])

for i in range(0,20):
  for j in range(0,84):
    vali.append(v[i][j])

trim=[]
valim=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

for i in range(0,80):
  gt=vali[78*i:78*(i+1)]
  mt=np.mean(gt)
  trim.append(mt)

for i in range(0,20):
  gv=vali[84*i:84*(i+1)]
  mv=np.mean(gv)
  valim[i]=mv

plt.plot(range(1,len(tiri)+1),tiri)
s_loss=(len(tiri))/(len(valim))
a_loss=2
d_loss=[2]
for id in range(1,len(valim)):
  d_loss.append(a_loss+s_loss)
  a_loss+=s_loss
plt.plot(d_loss,valim)
plt.xlabel('epochs')
plt.ylabel('loss')
plt.xticks([0,310,620,930,1240,1550,1860,2170,2480,2790,3100,3410,3720,4030,4340,4650,4960,5270,5580,5890],
           ['1', '2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'])
plt.ylim(0,0.05)
plt.legend(['train','val'])

################################################################################
# Define Functions For FCLF-CNN And Run And Save Results
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

## Function To Training FCLF-CNN

In [ ]:
def train_fclf(dataset, model1,model2, loss_fn1,loss_fn2, optimizer1,optimizer2,batch_size,epoch):

  model1.eval()

  vol=len(dataset)
  iter_s = vol // batch_size
  iter_f = vol % batch_size
  
  number_batch=0
  
  loss_all_lst=[]
  yd_all=[]
  y_all=[]

  for idx in range(0,iter_s):
    number_batch=int(idx)+1
    print('epoch=' ,epoch ,'batch = ',number_batch)
    batch_images=custom_data(root='/content',book=dataset,size=batch_size)
    images,lables=batch_images._get()
    dataset=batch_images._remove()
    data_loader=data.DataLoader(dataset=images,batch_size=batch_size)

    for (x,y) in data_loader:
      model2.train()

      # Compute prediction error f
      pred,x = model1(x)
      loss1 = loss_fn1(pred, x)
      pred=pred.reshape(len(x),3,256,256)

      # Compute prediction error cnn
      yp = model2(pred.clone())
      loss2 = loss_fn2(yp, y)
      loss_all=loss1+loss2

      optimizer2.zero_grad()
      loss2.backward(inputs=list(model2.parameters()))
      optimizer2.step()

      # get accuracy
      yd=(torch.max(yp,1)[1])
      print('(',yd,')','|' , '(',y,')')

      num_corrects=torch.sum(torch.max(yp,1)[1]==y)
      acc = num_corrects.float()/float(batch_size)

      loss_all_lst.append(loss_all.detach().cpu().numpy().item())
      yd_all.append(yd.cpu().numpy())
      y_all.append(y.cpu().numpy())
  
      print('num_corrects:' , num_corrects.item(),', Train Loss_f: ', loss1.item(), ', Train Loss_c: ', loss2.item(),
            ', Train Loss all:', loss_all.item() , ', Train Accuracy:' , acc.item())
########################################################################################################
  if iter_f > 0 :
    print('epoch=' ,epoch ,'last batch = ' ,number_batch+1)
    batch_images=custom_data(root='/content',book=dataset,size=iter_f)
    images,lables=batch_images._get()
    dataset=batch_images._remove()
    data_loader=data.DataLoader(dataset=images,batch_size=iter_f)

    for (x,y) in data_loader:
      model2.train()

      # Compute prediction error f
      pred,x = model1(x)
      loss1 = loss_fn1(pred, x)
      pred=pred.reshape(len(x),3,256,256)

      # Compute prediction error cnn
      yp = model2(pred.clone())
      loss2 = loss_fn2(yp, y)

      loss_all=loss1+loss2

      # Backpropagation
      optimizer2.zero_grad()
      loss2.backward(inputs=list(model2.parameters()))
      optimizer2.step()

      # get accuracy
      yd=(torch.max(yp,1)[1])
      print('(',yd,')','|' , '(',y,')')

      num_corrects=torch.sum(torch.max(yp,1)[1]==y)
      acc = num_corrects.float()/float(batch_size)

      loss_all_lst.append(loss_all.detach().cpu().numpy().item())
      yd_all.append(yd.cpu().numpy())
      y_all.append(y.cpu().numpy())
  
      print('num_corrects:' , num_corrects.item(),', Train Loss 1:', loss1.item(), ', Train Loss 2:', loss2.item(),
            ', Train Loss all:', loss_all.item() , ', Train Accuracy:' , acc.item())
      
  return  loss_all_lst , yd_all , y_all

## Function To Validation and Testing FCLF-CNN

In [ ]:
def test_fclf(dataset , model1 , model2 , loss_fn1 , loss_fn2 , batch_size , epoch):
  model1.eval()
  model2.eval()

  vol=len(dataset)
  iter_s = vol // batch_size
  iter_f = vol % batch_size
  
  number_batch=0

  loss1_lst=[]
  loss2_lst=[]
  loss_all_lst=[]
  yd_all=[]
  y_all=[]

  loss_value=0
  for idx in range(0,iter_s):
    number_batch=int(idx)+1
    print('epoch=' ,epoch ,'batch = ',number_batch)
    batch_images=custom_data(root='/content',book=dataset,size=batch_size)
    images,lables=batch_images._get()
    dataset=batch_images._remove()
    data_loader=data.DataLoader(dataset=images,batch_size=batch_size)
 
    with torch.no_grad():
     for (x,y) in data_loader:

      # Compute prediction error f
      pred,x = model1(x)
      loss1 = loss_fn1(pred, x)
      pred=pred.reshape(len(x),3,256,256)

      # Compute prediction error cnn
      yp = model2(pred)
      loss2 = loss_fn2(yp, y)

      loss_all=loss1+loss2

      # get accuracy
      yd=(torch.max(yp,1)[1])
      print('(',yd,')','|' , '(',y,')')

      num_corrects=torch.sum(torch.max(yp,1)[1]==y)
      acc = num_corrects.float()/float(batch_size)

      # make lists
      loss1_lst.append(loss1.detach().cpu().numpy().item())
      loss2_lst.append(loss2.detach().cpu().numpy().item())
      loss_all_lst.append(loss_all.detach().cpu().numpy().item())
      yd_all.append(yd.cpu().numpy())
      y_all.append(y.cpu().numpy())
  
      print('num_corrects:' , num_corrects.item(),', val Loss 1:', loss1.item(), ', val Loss 2:', loss2.item(),
            ', val Loss all:', loss_all.item() , ', val Accuracy:' , acc.item())
########################################################################################################
  if iter_f > 0 :
    print('epoch=' ,epoch ,'last batch = ' ,number_batch+1)
    batch_images=custom_data(root='/content',book=dataset,size=iter_f)
    images,lables=batch_images._get()
    dataset=batch_images._remove()
    data_loader=data.DataLoader(dataset=images,batch_size=iter_f)
    
    with torch.no_grad():
     for (x,y) in data_loader:

      # Compute prediction error f
      pred,x = model1(x)
      loss1 = loss_fn1(pred, x)
      pred=pred.reshape(len(x),3,256,256)

      # Compute prediction error cnn
      yp = model2(pred)
      loss2 = loss_fn2(yp, y)

      loss_all=loss1+loss2

      # get accuracy
      yd=(torch.max(yp,1)[1])
      print('(',yd,')','|' , '(',y,')')

      num_corrects=torch.sum(torch.max(yp,1)[1]==y)
      acc = num_corrects.float()/float(batch_size)

      # make lists
      loss1_lst.append(loss1.detach().cpu().numpy().item())
      loss2_lst.append(loss2.detach().cpu().numpy().item())
      loss_all_lst.append(loss_all.detach().cpu().numpy().item())
      yd_all.append(yd.cpu().numpy())
      y_all.append(y.cpu().numpy())
  
      print('num_corrects:' , num_corrects.item(),', val Loss 1:', loss1.item(), ', val Loss 2:', loss2.item(),
            ', val Loss all:', loss_all.item() , ', val Accuracy:' , acc.item())

  return  loss2_lst , yd_all , y_all

## Load CNN And Fully Connected Layer First parameters

In [ ]:
model_f.load_state_dict(torch.load('/content/drive/MyDrive/modelsaved/saved_model_model_f20.pth'))
model_effi_b4=load_model (model_effi_b4 ,'efficient_b4_result',19)

## Run FCLF-CNN And Save Models And Results

In [ ]:
name='3*1024_fTrained_cnnTrained_fFix'
epoch=1

tr_loss_all , tr_yd , tr_y =[],[],[]
val_loss_all , val_yd , val_y =[],[],[]

# train and val
for epo in range(0,epoch):

    print(' ')
    print('epoch ========================================================================================= ',epo+1)
    print(' ')
    print('trainnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn')
    print(' ')

    tr_loss_e_all , tr_yd_e , tr_y_e = train_fclf(train_Book, model_f , model_effi_b4 , loss_fn_mse , loss_fn_ce , optimizer_f , optimizer_effi_b4 ,16, epo+1)
    print(' ')
    print('train=',torch.cuda.memory_allocated(device=dev))

    torch.save(model_effi_b4.state_dict(), '/content/drive/MyDrive/modelsaved/saved_model_fclf'+name+'_epoch='+str(epo+1)+'.pth')

    print('valllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllllll')
    print(' ')

    val_loss_e_all , val_yd_e , val_y_e = test_fclf(val_Book, model_f , model_effi_b4 , loss_fn_mse , loss_fn_ce ,16, epo+1)

    tr_loss_all.append(tr_loss_e_all)
    tr_yd.append(tr_yd_e)
    tr_y.append(tr_y_e)

    val_loss_all.append(val_loss_e_all)
    val_yd.append(val_yd_e)
    val_y.append(val_y_e)

np.savez('/content/drive/MyDrive/train_result_fclf_'+name   , tr_loss_all , tr_yd , tr_y)
np.savez('/content/drive/MyDrive/val_result_fclf_'+name     , val_loss_all , val_yd , val_y)
  
print('Done')

## Plot FCLF-CNN Result

### Load Training And Evaluation Results

In [ ]:
train_acc_fclf , train_loss_fclf , val_acc_fclf , val_loss_fclf = acc_mat_func ('fclf_3*1024_fTrained_cnnTrained_fFix',1239,78,1344,84)

### Plot Accuracy

In [ ]:
print(len(train_acc_fclf))
print(len(val_acc_fclf))
plt.plot(range(1,len(train_acc_fclf)+1),train_acc_fclf)
s_acc=(len(train_acc_fclf))/(len(val_acc_fclf))
a_acc=2.5
d_acc=[2.5]
for id in range(1,len(val_acc_fclf)):
  d_acc.append(a_acc+s_acc)
  a_acc+=s_acc
plt.plot(d_acc,val_acc_fclf)
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.ylim(0,101)
plt.xticks([2.5,6.5,10.5,14.5,18.5,22.5,26.5,30.5,34.5,38.5,42.5,46.5,50.5,54.5,58.5,62.5,66.5,70.5,74.5,78.5],
           ['1', '2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'])
plt.legend(['train','val'])

### Plot Loss

In [ ]:
plt.plot(range(1,len(train_loss_effi)+1),train_loss_effi)
print(len(train_loss_effi))
print(len(val_loss_effi))
s_loss=(len(train_loss_effi))/(len(val_loss_effi))
a_loss=2
d_loss=[2]
for id in range(1,len(val_loss_effi)):
  d_loss.append(a_loss+s_loss)
  a_loss+=s_loss
plt.plot(d_loss,val_loss_effi)
plt.xlabel('epochs')
plt.ylabel('loss')
plt.xticks([2,5,8,11,14,17,20,23,26,29,32,35,38,41,44,47,50,53,56,59],
           ['1', '2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20'])
plt.ylim(0,5)
plt.legend(['train','val'])

## Test FCLF-CNN And Plot Confusion Matrix

In [ ]:
model_f.load_state_dict(torch.load('/content/drive/MyDrive/modelsaved/saved_model_model_f1.pth'))
model_effi_b4.load_state_dict(torch.load('/content/drive/MyDrive/modelsaved/saved_model_fclf3*1024_fTrained_cnnTrained_fFix_epoch=1.pth'))

test_loss_h , test_yd_h , test_y_h =[],[],[]
test_loss_e , test_yd_e , test_y_e = test_fclf(test_Book, model_f , model_effi_b4 , loss_fn_mse , loss_fn_ce ,16, 1)

test_loss_h.append(test_loss_e)
test_yd_h.append(test_yd_e)
test_y_h.append(test_y_e)

 ##########################################################################    test
print('#################################################################    test   ###################################################################') 

number_test_loss_epoch=len(test_loss_h[0])
number_test_epoch=len(test_yd_h)
number_test_batch=len(test_yd_h[0])-1
number_test_batchsize=16
number_test_endbatch= 2100 % number_test_batchsize
if (number_test_endbatch==0):
  number_test_endbatch=16

test_loss=[]
test_yd=[]
test_y=[]

batch_fix=0
for epoch in range(0,1):
    for batch in range(0,number_test_batch):
      
      batch_fix=batch+1
      for size in range(0,number_test_batchsize):
        test_yd.append(test_yd_h[epoch][batch][size])
        test_y.append(test_y_h[epoch][batch][size])

    for size in range(0,number_test_endbatch):
      test_yd.append(test_yd_h[epoch][batch_fix][size])
      test_y.append(test_y_h[epoch][batch_fix][size])

number_a_epoch=1
number_test_point=1

test_matrix=torch.tensor([[0,0,0],[0,0,0],[0,0,0]])

num_test=0
for id in range(0,len(test_yd)):
    test_yd_b=torch.tensor(test_yd[id])
    test_y_b=torch.tensor(test_y[id])
    # Matrix
    if(test_yd_b==0 and test_y_b==0):
      test_matrix[0][0]+=1;
    elif(test_yd_b==0 and test_y_b==1):
      test_matrix[0][1]+=1;
    elif(test_yd_b==0 and test_y_b==2):
      test_matrix[0][2]+=1;
    elif(test_yd_b==1 and test_y_b==0):
      test_matrix[1][0]+=1;
    elif(test_yd_b==1 and test_y_b==1):
      test_matrix[1][1]+=1;
    elif(test_yd_b==1 and test_y_b==2):
      test_matrix[1][2]+=1;
    elif(test_yd_b==2 and test_y_b==0):
      test_matrix[2][0]+=1;
    elif(test_yd_b==2 and test_y_b==1):
      test_matrix[2][1]+=1;
    elif(test_yd_b==2 and test_y_b==2):
      test_matrix[2][2]+=1;

    num_corrects_test=torch.sum(test_yd_b==test_y_b)
    num_test+=num_corrects_test

    acc_test=num_test/len(test_yd)*100

sumy=0
mean=0
for id in range(0,number_test_batch):

    losses_test = test_loss_h[0][id]*16
    sumy = sumy + losses_test

if (number_test_endbatch>0):
    losses_test = test_loss_h[0][id+1]*number_test_endbatch
    sumy = sumy + losses_test

mean = sumy / len(test_yd)
test_loss_all=mean

print('num_test',num_test)
print('acc_test',acc_test)
print('test_loss_all',test_loss_all)
print('test_matrix')
print(test_matrix)